In [1]:
!git clone https://github.com/timothycao/agnews-classifier.git
%cd agnews-classifier

Cloning into 'agnews-classifier'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 17), reused 28 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 10.68 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/kaggle/working/agnews-classifier


In [19]:
# Model

from peft import LoraConfig
from model import create_lora_model

lora_config = LoraConfig(
    r=2,
    lora_alpha=6,
    lora_dropout=0.1,
    bias='none',
    target_modules=['query', 'value', 'key'],
    task_type='SEQ_CLS',
    use_rslora = False # regular LoRA: lora_alpha/r, rs-LoRA: lora_alpha/sqrt(r) -> normalization technique
)

model = create_lora_model(lora_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 704,260 || all params: 125,352,968 || trainable%: 0.5618


In [20]:
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): Mod

In [21]:
# Training

from transformers import TrainingArguments
from train import main as train

training_args = TrainingArguments(
    # Core training configs
    max_steps=2000,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    optim='adamw_torch',
    learning_rate=5e-5,
    lr_scheduler_type='linear',
    
    # Logging, evaluation, and checkpointing
    output_dir='/kaggle/working/saved_models',
    logging_strategy='steps',
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,

    # Miscellaneous
    report_to='none',
    dataloader_num_workers=4,
    gradient_checkpointing=False,
    gradient_checkpointing_kwargs={'use_reentrant':True}
)

train(model, training_args)

Starting training...


Step,Training Loss,Validation Loss,Accuracy
100,1.369400,1.330230,0.557813
200,1.107900,0.539349,0.879687
300,0.416700,0.350651,0.879687
400,0.321200,0.347378,0.881250
500,0.321400,0.327569,0.903125
600,0.308600,0.343952,0.889062
700,0.314600,0.326894,0.898438
800,0.324800,0.312527,0.906250
900,0.295500,0.317506,0.907813
1000,0.293600,0.313076,0.906250


In [23]:
# Inference

from inference import main as inference

data_path = '/kaggle/input/deep-learning-spring-2025-project-2/test_unlabelled.pkl'
checkpoint = '/kaggle/working/saved_models/checkpoint-2000'
output_dir = '/kaggle/working/saved_predictions'

inference(data_path, checkpoint, output_dir)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running inference...


100%|██████████| 250/250 [00:59<00:00,  4.22it/s]

Predictions saved to /kaggle/working/saved_predictions/predictions_checkpoint-2000.csv


In [24]:
import pandas as pd

csv_path = '/kaggle/working/saved_models/checkpoint-2000/processed_log_history.csv'

df = pd.read_csv(csv_path)
df

,Step,Train Loss,Test Loss,Train Acc,Test Acc,Learning Rate,Epochs,Loss Spread,Loss Ratio,Acc Spread,Acc Ratio
0,100,1.3694,1.330230,0.351875,0.557813,0.000048,0.013405,0.039170,1.029446,-0.205937,0.630812
1,200,1.1079,0.539349,0.680804,0.879687,0.000045,0.026810,0.568551,2.054143,-0.198884,0.773915
2,300,0.4167,0.350651,0.873661,0.879687,0.000043,0.040214,0.066049,1.188361,-0.006027,0.993149
3,400,0.3212,0.347378,0.887946,0.881250,0.000040,0.053619,-0.026178,0.924640,0.006696,1.007599
4,500,0.3214,0.327569,0.889732,0.903125,0.000037,0.067024,-0.006169,0.981166,-0.013393,0.985171
5,600,0.3086,0.343952,0.900893,0.889062,0.000035,0.080429,-0.035352,0.897218,0.011830,1.013307
6,700,0.3146,0.326894,0.896429,0.898438,0.000033,0.093834,-0.012294,0.962390,-0.002009,0.997764
7,800,0.3248,0.312527,0.891964,0.906250,0.000030,0.107239,0.012273,1.039270,-0.014286,0.984236
8,900,0.2955,0.317506,0.899554,0.907813,0.000028,0.120643,-0.022006,0.930691,-0.008259,0.990902
9,1000,0.2936,0.313076,0.900893,0.906250,0.000025,0.134048,-0.019476,0.937792,-0.005357,0.994089
